In [ ]:
%matplotlib inline
import os, sys
from IPython.display import clear_output

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# python wrapper for cpp code
from qihmc import Ising

# MDS problem to qubo map (install the other repo)
from qlp.mds.qubo import get_mds_qubo, QUBO_to_Ising
from qlp.mds.graph_tools import get_plot_mpl

# import qlp.mds.graph_tools as gt
from qlp.mds import graph_tools as gt

# from qlp.mds.qubo import get_mds_qubo
# from qlp.mds.mds_qlpdb import QUBO_to_Ising

from qlpdb.graph.models import Graph as graph_Graph
from qlpdb.experiment.models import DWaveExperiment as experiment_Experiment
from qlpdb.data.models import Data as data_Data

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

Below is my simple hack at a progress bar.  I need this so that I can gauge if my runs got stuck in a local minimum or not. . .

In [ ]:
# my simple progress bar. . .
def update_progress(progress):
    bar_length = 40
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait=True)
    text = "Progress: [{0}] {1:.1f}%".format(
        "#" * block + "-" * (bar_length - block), progress * 100
    )
    print(text)

In [ ]:
# available graphs--choose one of these options
# vertices = 4, 10, 50, 100
total_vertices = 10

In [ ]:
# this makes some pretty pictures
graphs = graph_Graph.objects.filter(total_vertices=total_vertices)
for idx, graph in enumerate(graphs):
    # plot graph
    adjacency_map = graph.adjacency
    gt.get_plot(adjacency_map, directed=False)
    print(graph.id)
    plt.show()

In [ ]:
# Choose graph to read out (these numbers are next to each graph figure above)
graph_id = 6

In [ ]:
# This gets info about the D-Wave run for this particular graph
for idx, experiment in enumerate(
    experiment_Experiment.objects.filter(graph_id=graph_id)
):
    experiment_id = experiment.id
    print(experiment_id)
    print(experiment)


# Print QUBO and Ising information for this system
penalty = float(experiment_Experiment.objects.get(id=experiment_id).p)
adjacency_map = {tuple(edge) for edge in graph_Graph.objects.get(id=graph_id).adjacency}
qubo = get_mds_qubo(
    adjacency_map, penalty=penalty, directed=False, triangularize=True, dtype="d"
)
print("QUBO formulation")
print(qubo.todense())
J, h, c = QUBO_to_Ising(qubo.todense().tolist())
print("Ising formulation")
print(J)
print(h)
print(c)
offset = c + penalty * total_vertices
# data_Data.objects.filter(experiment_id = experiment_id)

Add parameters used for ising model

In [ ]:
init_beta = 0.2  # this is the initial hot temperature to start the annealing process
beta = 2.0  # this is the final 'cold' temperature
md_steps = 10  # for 4 vertices use 8, 10 vertices use 10, 50 vertices use 22, 100 vertices use 30
ergodicity_jumps = -100  # negative here means that we do not do any ergodicity jumps
save_frequency = 10  # this really means that we take measurements every 10th trajectory during production

ising = Ising(J, h, offset, beta, md_steps, ergodicity_jumps)

Now do the runs . . .

In [ ]:
# arrays to store the obserables
energies = []
contEnergies = []
spins = []
contSpins = []

n_therm = 4000  # this is the number of 'cooling' steps
numOfMeasurements = 1000  # self-explanatorz
n_trajectories = 1000  # number of trajectories per measurement

currentMeasurement = 0
while currentMeasurement < numOfMeasurements:
    # first cool from initial 'hot' temperature
    ising.anneal(init_beta, beta, n_therm, md_steps)

    # now due production run at target 'cold' temperature
    ising.run_hmc(beta, n_trajectories, md_steps, ergodicity_jumps, save_frequency)

    # now I calculate some obserables
    psi = np.mean(ising.configs, axis=0) / np.sqrt(ising.beta) - ising.k
    pgs = np.floor(psi + 0.5)  # this takes the spins to their nearest integer
    norm = pgs @ pgs

    # sometimes the solution is not a valid spin solution, so here I check for valid spin solutions
    if np.floor(norm + 0.5) == len(
        pgs
    ):  # if valid, I store solution and its obserables
        energies.append(pgs @ J @ pgs + h @ pgs + offset)
        spins.append(pgs)
        contEnergies.append(np.mean(ising.energy))
        contSpins.append(psi)
        currentMeasurement += 1
        update_progress(currentMeasurement / numOfMeasurements)

    # I repeat until I have number of measurements = numOfMeasurements
update_progress(1)

# here I save the data into text files (yes, I know, not sophisticated at all)
# np.savetxt('energies'+str(total_vertices)+'_vertices_graphID_'+str(graph_id)+'.txt',np.array(energies))
# np.savetxt('spins'+str(total_vertices)+'_vertices_graphID_'+str(graph_id)+'.txt',np.array(spins))
# np.savetxt('contEnergies'+str(total_vertices)+'_vertices_graphID_'+str(graph_id)+'.txt',np.array(contEnergies))
# np.savetxt('contSpins'+str(total_vertices)+'_vertices_graphID_'+str(graph_id)+'.txt',np.array(contSpins))

In [ ]:
# here I print out some info on the lowest energy state that was calculated
egs = min(energies)
numOfEgs = 0
egs_indices = []
for i in range(len(energies)):
    if energies[i] == egs:
        numOfEgs += 1
        egs_indices.append(i)
print("# Egs = ", egs, " happened ", numOfEgs, " times")
print("# Prob. of Success = ", numOfEgs * 1.0 / len(energies))

# here is an example of the spin configuration for the lowest energy found
randomIndex = np.random.randint(
    len(egs_indices)
)  # I choose some random low energy state
plt.stem(
    np.array([i for i in range(ising.Lambda)]),
    spins[egs_indices[randomIndex]],
    use_line_collection=True,
)

plt.xlabel(r"$i$")
plt.ylabel(r"$S_i$")

In [ ]:
# the histogram of the data
energy = dict()
for row in data_Data.objects.filter(experiment_id=experiment_id):
    if row.energy in energy:
        energy[row.energy] += 1
    else:
        energy[row.energy] = 1
x = np.sort(list(energy.keys()))
y = [energy[i] for i in x]

fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(10, 5))

ax[0].hist(
    energies,
    [-0.5 + i for i in range(int(min(energies)), int(max(energies)) + 1)],
    edgecolor="Green",
    density=1,
    facecolor="green",
    alpha=0.75,
)
ax[1].bar(x=x, height=np.array(y) / (1.0 * sum(y)))

ax[0].set_xlabel(r"$E_i$")
ax[0].set_title(r"HMC")
ax[1].set_xlabel(r"$E_i$")
ax[1].set_title(r"D-Wave")
ax[0].set_ylabel("Probability")
ax[0].grid(True)
ax[1].grid(True)

# plt.savefig(str(total_vertices)+'_vertices_graphID_'+str(graph_id)+'.png')

for idx, graph in enumerate(graphs):
    # plot graph
    if graph_id == graph.id:
        adjacency_map = graph.adjacency
        gt.get_plot(adjacency_map, directed=False)
        break

# plt.savefig(str(total_vertices)+'_vertices_graphID_'+str(graph_id)+'_connectivity.png')

# CK additions for the filter

For the MDS problem, we intend to find the least amount of nodes fulfilling
$$
\boldsymbol{x}_{0}=\arg \min _{x}\left(\sum_{i=1}^N x_{i}\right)
$$
under the MDS constraints, where $N$ is the total number of vertices and the node $x_i$ is colored if it is equal to one, else zero.

Mapping this to the QUBO notation results in
$$
\begin{aligned}
\chi^{2}(\boldsymbol{\Psi}) &=\boldsymbol{\Psi}^{T}\left(\begin{array}{ll}
Q_{x x} & Q_{x s} \\
Q_{s x} & Q_{s s}
\end{array}\right) \boldsymbol{\Psi}+p\|\boldsymbol{b}\|^{2} \\
& \equiv \boldsymbol{\Psi}^{T} Q \boldsymbol{\Psi}+C
\end{aligned}
$$
with
$$
\begin{aligned}
Q_{x x} &=1+p\left[J^{T} J+J^{T}+J-\operatorname{diag}\left(J^{T} \mathbf{1}+\mathbf{1}^{T} J\right)-1\right] \\
Q_{s x} &=-p(1+J)^{T} T_{s} \\
Q_{s s} &=p\left[T_{s}^{T} T_{s}+\operatorname{diag}\left(T_{s}^{T} \mathbf{1}+\mathbf{1}^{T} T_{s}\right)\right] \\
C &=p N
\end{aligned}
$$
where $J$ is the graph adjacency matrix and $T$ maps integer variables to qubits.

If the MDS constraints are fulfilled, the objective function $\chi^{2}(\boldsymbol{\Psi})$ is independent of $p$.
Thus, the energy is equal to $\boldsymbol{\Psi}^{T} \boldsymbol{\Psi}$.
This is what I am checking below.

If I understand your code properly, we should obtain back the qubits from the spins by mapping $\uparrow = 1 \to 1$ and $\downarrow = -1 \to 0$.

In [ ]:
n = 2
s0 = spins[n]
q0 = (s0 + 1) / 2
e0 = energies[n]
print(energies[1])

In [ ]:
print(graph_id, experiment_id)
penalty = float(experiment_Experiment.objects.get(id=experiment_id).p)
adjacency_map = {tuple(edge) for edge in graph_Graph.objects.get(id=graph_id).adjacency}
qubo = get_mds_qubo(
    adjacency_map, penalty=penalty, directed=False, triangularize=True, dtype="d"
)

The energy in the QUBO basis is nothing else than $\vec q \cdot Q \cdot q + p N$, where $\vec q$ is a qubit vector, $Q$ the QUBO, $p$ the penalty term and $N$ the number of vertices.

In [ ]:
assert q0.T @ qubo @ q0 + penalty * total_vertices == e0

As a cross-check we should also obtain the same anergy using the Ising Hamiltonian

In [ ]:
J, h, c = QUBO_to_Ising(qubo.todense().tolist())
offset = c + penalty * total_vertices

assert s0.T @ J @ s0 + h @ s0 + offset == e0

The below function provides a filter for checking if a given solution fulfills the MDS condition and returns as mask (indices if you like) to filter out wrong solutions

In [ ]:
def get_valid_dominating_set_mask(ss, ee, nn):
    """Identify spin configurations which satisfy MDS constraints.

    Checks if the objective function (sum over all vertices) equals the energy
    (and thus the constraint is zero).
    Also works for qubits as inputs.
    """
    ss = np.array(ss, dtype=int)
    ee = np.array(ee, dtype=int)
    qq = (ss + 1) // 2
    return qq[:, :nn].sum(axis=1) == ee

In [ ]:
mask = get_valid_dominating_set_mask(spins, energies, total_vertices)
print(f"Number of valid dominating sets: {mask.sum()}/{len(energies)}")

In [ ]:
idx0 = np.array(energies)[mask].argmin()

s_valid = np.array(spins)[mask][idx0]
# get indices corresponding to vertices (first total_vertices entries) which point up
nodes_index = [n for n, el in enumerate(s_valid[:total_vertices]) if el == 1]

gt.get_plot(
    graph_Graph.objects.get(id=graph_id).adjacency,
    directed=False,
    color_nodes=nodes_index,
    seed=13,
)
plt.show()

You can do the same thing for the db entries

In [ ]:
# get all the spin configs and energies for a given experiment
db_data = data_Data.objects.filter(experiment_id=experiment_id).values_list(
    "spin_config", "energy"
)
# Map them to arrays (this is a funny looking transpose for lists)
db_qubits, db_energies = map(np.array, zip(*db_data))
# And generate the mask as before. Also works for qubits as (0+1)//2 = 0
db_mask = get_valid_dominating_set_mask(db_qubits, db_energies, total_vertices)
print(f"Number of valid dominating sets: {db_mask.sum()}/{len(db_energies)}")

In [ ]:
idx0 = np.array(db_energies)[mask].argmin()

db_q_valid = db_qubits[mask][idx0]
# get indices corresponding to vertices (first total_vertices entries) which point up
nodes_index = [n for n, el in enumerate(db_q_valid[:total_vertices]) if el == 1]

gt.get_plot(
    graph_Graph.objects.get(id=graph_id).adjacency,
    directed=False,
    color_nodes=nodes_index,
    seed=13,
)
plt.show()